<a href="https://colab.research.google.com/github/Pathway2008/law_predict/blob/main/XLnet_ELECTRA_wight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.7 MB/s eta 0:00:00


In [4]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.6 MB/s eta 0:00:00


In [1]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/law/train.csv')
test = pd.read_csv('/content/drive/MyDrive/law/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/law/sample_submission.csv')

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import BCEWithLogitsLoss
from tqdm import tqdm

In [3]:
train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

In [6]:
train_df['first_party_winner'].value_counts()

1    1321
0     661
Name: first_party_winner, dtype: int64

In [12]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', max_length=512)
train_encodings = tokenizer(list(train_df['facts']), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(list(val_df['facts']), truncation=True, padding=True, max_length=512)

In [13]:
train_labels = torch.tensor(list(train_df['first_party_winner']))
val_labels = torch.tensor(list(val_df['first_party_winner']))

In [14]:
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']), train_labels)
val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']),
                            torch.tensor(val_encodings['attention_mask']), val_labels)

In [15]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [16]:
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [18]:
optimizer = AdamW(model.parameters(), lr=2e-5)
weights = torch.tensor([2.0, 1.0])  # Class weights for 0 and 1 classes respectively
loss_fn = BCEWithLogitsLoss(weight=weights.to(device))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0

    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for batch in progress_bar:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'Loss': loss.item()})

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}')

Epoch 1/5 - Loss: 0.6531


Epoch 2/5 - Loss: 0.6435


Epoch 3/5 - Loss: 0.6392


Epoch 4/5 - Loss: 0.6386


Epoch 5/5 - Loss: 0.5963


In [20]:
model.eval()
val_loss = 0
val_accuracy = 0

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        val_loss += loss.item()

        predictions = torch.argmax(logits, dim=1)
        val_accuracy += torch.sum(predictions == labels).item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = val_accuracy / len(val_dataset) * 100

    print(f'Validation Loss: {avg_val_loss:.4f}')
    print(f'Validation Accuracy: {val_accuracy:.2f}%')

Validation Loss: 0.6620
Validation Accuracy: 63.71%


In [21]:
test_encodings = tokenizer(list(test['facts']), truncation=True, padding=True, max_length=512)

In [22]:
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                             torch.tensor(test_encodings['attention_mask']))

In [23]:
batch_size = 16
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [24]:
model.eval()

predictions = []

# Iterate over the test DataLoader for inference
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)

        # Pass the input tensors to the model for inference
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Obtain the predicted probabilities or labels
        probabilities = torch.sigmoid(logits)
        predicted_labels = torch.argmax(logits, dim=1)

        # Append the predictions to the list
        predictions.extend(predicted_labels.cpu().numpy())

In [25]:
sub['first_party_winner'] = predictions
sub['first_party_winner'].value_counts()

1    1156
0      84
Name: first_party_winner, dtype: int64

In [ ]:
# sigmoid가 아닌 softmax라면?

Electra

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import ElectraTokenizer, ElectraForSequenceClassification, AdamW
from torch.nn import CrossEntropyLoss
from sklearn.metrics import accuracy_score

In [6]:
tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')
train_encodings = tokenizer(list(train_df['facts']), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(list(val_df['facts']), truncation=True, padding=True, max_length=512)

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1982 entries, 2464 to 860
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  1982 non-null   object
 1   first_party         1982 non-null   object
 2   second_party        1982 non-null   object
 3   facts               1982 non-null   object
 4   first_party_winner  1982 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 157.4+ KB


In [10]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 1753 to 2423
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  496 non-null    object
 1   first_party         496 non-null    object
 2   second_party        496 non-null    object
 3   facts               496 non-null    object
 4   first_party_winner  496 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 23.2+ KB


In [ ]:
# Tokenize and encode the text data
tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=512,
                                         padding='max_length', return_tensors='pt') for text in train_texts]

input_ids = torch.cat([text['input_ids'] for text in tokenized_texts], dim=0)
attention_masks = torch.cat([text['attention_mask'] for text in tokenized_texts], dim=0)
labels = torch.tensor(train_labels)

In [17]:
train_texts = train_df['facts'].tolist()
train_df_labels = train_df['first_party_winner'].tolist()

tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=512,
                                         padding='max_length', return_tensors='pt') for text in train_texts]

train_input_ids = torch.cat([text['input_ids'] for text in tokenized_texts], dim=0)
train_attention_masks = torch.cat([text['attention_mask'] for text in tokenized_texts], dim=0)
train_labels = torch.tensor(train_df_labels)

In [18]:
val_texts = val_df['facts'].tolist()
val_labels = val_df['first_party_winner'].tolist()

tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=512,
                                         padding='max_length', return_tensors='pt') for text in val_texts]

val_input_ids = torch.cat([text['input_ids'] for text in tokenized_texts], dim=0)
val_attention_masks = torch.cat([text['attention_mask'] for text in tokenized_texts], dim=0)
val_labels = torch.tensor(val_labels)

In [19]:
# Create PyTorch datasets and dataloaders
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [20]:
model_elec = ElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator', num_labels=2)
optimizer = AdamW(model_elec.parameters(), lr=2e-5)

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.d

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_elec.to(device)

In [22]:
loss_fn = CrossEntropyLoss(weight=torch.tensor([1.0, 2.0]).to(device))

In [24]:
from tqdm import tqdm

epochs = 5

for epoch in range(epochs):
    model_elec.train()
    total_loss = 0

    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for batch in progress_bar:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model_elec(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'Loss': loss.item()})

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}')

Epoch 1/5 - Loss: 0.6458


Epoch 2/5 - Loss: 0.6314


Epoch 3/5 - Loss: 0.6130


Epoch 4/5 - Loss: 0.5430


Epoch 5/5 - Loss: 0.3048


In [25]:
model_elec.eval()
val_loss = 0
predictions = []
true_labels = []

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model_elec(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        val_loss += loss.item()

        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_dataloader)
    accuracy = accuracy_score(true_labels, predictions) * 100

print(f'Validation Loss: {avg_val_loss:.4f}')
print(f'Accuracy: {accuracy:.2f}%')

Validation Loss: 1.0621
Accuracy: 62.50%


In [26]:
test_texts = test['facts'].tolist()

In [27]:
tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=512,
                                         padding='max_length', return_tensors='pt') for text in test_texts]

In [28]:
input_ids = torch.cat([text['input_ids'] for text in tokenized_texts], dim=0)
attention_masks = torch.cat([text['attention_mask'] for text in tokenized_texts], dim=0)

In [29]:
test_dataset = TensorDataset(input_ids,attention_masks)

In [30]:
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [31]:
# Set the model to evaluation mode
model_elec.eval()

# List to store the predictions
predictions = []

# Iterate over the test dataloader
for batch in test_dataloader:
    batch_inputs = batch[0].to(device)
    batch_masks = batch[1].to(device)

    # Disable gradient calculation
    with torch.no_grad():
        # Forward pass
        outputs = model_elec(batch_inputs, attention_mask=batch_masks)
        logits = outputs.logits

        # Apply softmax to convert logits to probabilities
        probabilities = torch.softmax(logits, dim=1)

        # Get the predicted labels
        _, predicted_labels = torch.max(probabilities, dim=1)

        # Append the predicted labels to the predictions list
        predictions.extend(predicted_labels.cpu().tolist())

# Convert predictions to a numpy array if needed
predictions = np.array(predictions)

# Print the predicted labels
print(predictions)

[1 1 1 ... 1 1 1]


In [32]:
#softmax
sub['first_party_winner'] = predictions
sub['first_party_winner'].value_counts()

1    1056
0     184
Name: first_party_winner, dtype: int64

In [34]:
model_elec.eval()
predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)

        outputs = model_elec(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        batch_predictions = torch.argmax(logits, dim=1)
        predictions.extend(batch_predictions.cpu().numpy())

In [35]:
#just logits
sub['first_party_winner'] = predictions
sub['first_party_winner'].value_counts()

1    1056
0     184
Name: first_party_winner, dtype: int64

T5

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.nn import CrossEntropyLoss

In [4]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model_t5 = T5ForConditionalGeneration.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
train_encodings = tokenizer(list(train_df['facts']), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_df['facts']), truncation=True, padding=True, max_length=128)

In [10]:
train_labels = torch.tensor(list(train_df['first_party_winner']))
val_labels = torch.tensor(list(val_df['first_party_winner']))

In [11]:
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              train_labels)
val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']),
                            torch.tensor(val_encodings['attention_mask']),
                            val_labels)

In [12]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [13]:
model_t5.to(device)
optimizer = AdamW(model_t5.parameters(), lr=2e-5)
loss_fn = CrossEntropyLoss(weight=torch.tensor([2.0, 1.0]).to(device))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [8]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model_t5 = T5ForConditionalGeneration.from_pretrained('t5-small')

In [9]:
from torch.utils.data import DataLoader, Dataset
# Define the dataset class
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        facts = self.data.iloc[idx]['facts']
        first_party_winner = self.data.iloc[idx]['first_party_winner']

        input_text = "predict first_party_winner: " + facts
        target_text = str(first_party_winner)

        input_encoding = self.tokenizer.encode_plus(
            input_text,
            padding='max_length',
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )
        target_encoding = self.tokenizer.encode(
            target_text,
            padding='max_length',
            truncation=True,
            max_length=2,
            return_tensors='pt'
        )

        input_ids = input_encoding['input_ids'].squeeze()
        attention_mask = input_encoding['attention_mask'].squeeze()
        target_ids = target_encoding.squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'target_ids': target_ids
        }


In [10]:
train_dataset = CustomDataset(train_df, tokenizer)
val_dataset = CustomDataset(val_df, tokenizer)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [12]:
optimizer = AdamW(model_t5.parameters(), lr=2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
epochs = 5

for epoch in range(epochs):
    model_t5.train()
    total_loss = 0

    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)

        optimizer.zero_grad()

        outputs = model_t5(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids)
        loss = outputs.loss

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'Loss': loss.item()})

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}')

KeyboardInterrupt: ignored

In [ ]:
model_t5.eval()
val_loss = 0
val_accuracy = 0

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model_t5(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        val_loss += loss.item()

        predictions = torch.argmax(outputs.logits, dim=1)
        val_accuracy += torch.sum(predictions == labels).item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = val_accuracy / len(val_dataset) * 100

    print(f'Validation Loss: {avg_val_loss:.4f}')
    print(f'Validation Accuracy: {val_accuracy:.2f}%')